# Observer

This Observer notebook monitors USB cameras for changes on the DMA playing field.

In [1]:
import json
import cv2
from collections import defaultdict
import numpy as np
import imutils
from matplotlib import pyplot as plt
from pyzbar.pyzbar import decode as qrDecode
from math import cos, acos, degrees, radians
from time import sleep
from datetime import datetime
import requests
from dataclasses import dataclass
from traceback import format_exc

In [2]:
@dataclass
class CamCoordinate:
    x: int
    y: int
    

@dataclass
class RealCoordinate:
    x: float
    y: float

In [3]:
sr = cv2.dnn_superres.DnnSuperResImpl_create()
sr.readModel("FSRCNN_x4.pb")
sr.setModel("fsrcnn", 4);


def distanceFormula(pt0, pt1):
    return sum([(pt1[i] - pt0[i])**2 for i in range(len(pt0))]) ** 0.5


def capture_camera(cam_num):
    try:
        cam = cv2.VideoCapture(cam_num)
        retval, image = cam.read()
    finally:
        cam.release()
    retval, buff = cv2.imencode('.jpg', image)
    return buff


MAX_CAM_ID = 10


def identify_usb_cameras(device_numbers=list(range(MAX_CAM_ID))):
    functional = []
    for dn in device_numbers:
        try:
            img = capture_camera(dn)
            functional.append(dn)
        except Exception as e:
            continue
    return functional


def hStackImages(images):
    if len(images) == 0:
        return np.zeros((1, 1), dtype="uint8")
    baseIm = None
    maxHeight = max([im.shape[0] for im in images])
    for im in images:    
        if im.shape[0] < maxHeight:
            addition = np.zeros((maxHeight - im.shape[0], im.shape[1], 3), np.uint8)
            im = np.vstack((im, addition))
        if baseIm is None:
            baseIm = im
        else:
            baseIm = np.hstack((
                baseIm,
                np.zeros((maxHeight, 10, 3), np.uint8),
                im))
    return baseIm


def vStackImages(images):
    if len(images) == 0:
        return np.zeros((1, 1), dtype="uint8")
    baseIm = None
    maxWidth = max([im.shape[1] for im in images])
    for im in images:    
        if im.shape[0] < maxWidth:
            addition = np.zeros((im.shape[0], maxWidth - im.shape[1], 3), np.uint8)
            im = np.hstack((im, addition))
        if baseIm is None:
            baseIm = im
        else:
            baseIm = np.vstack((
                baseIm,
                np.zeros((10, maxWidth, 3), np.uint8),
                im))
    return baseIm

In [4]:
@dataclass
class CameraChange:
    camNum: int
    changePoints: np.array
    before: np.array
    after: np.array
    changeType: str = "unclassified"
    lastChange: object = None
    
    def __post_init__(self):
        if self.changeType is None:
            self.corner = None
            self.width = None
            self.height = None
            self.center = None
            self.before = None
            self.after = None
        else:
            xS = [pt[0] for pt in self.changePoints]
            yS = [pt[1] for pt in self.changePoints]
            minX = min(xS)
            minY = min(yS)
            self.corner = [minX, minY]
            self.width = max(xS) - minX
            self.height = max(yS) - minY
            self.center = [min(yS) + int(self.height / 2), min(xS) + int(self.width / 2)]
            self.before = self.before[int(minY):int(minY + self.height), int(minX):int(minX + self.width)]
            self.after = self.after[int(minY):int(minY + self.height), int(minX):int(minX + self.width)]
    
    @property
    def clipBox(self):
        return [int(i) for i in (*self.corner, self.width, self.height)]
    
    def classify(self, changeType: str, lastChange: object=None):
        assert changeType in ["add", "move", "delete", "unclassified"]
        self.lastChange = lastChange
        self.changeType = changeType

    def changeOverlap(self, change):
        zeros = np.zeros(cameras[self.camNum].mostRecentFrame.shape[:2], np.uint8)
        if self.changeType is None or change.changeType is None:
            return zeros.any()

        allPoints = [pt for pt in self.changePoints] + [pt for pt in change.changePoints]
        xS = [pt[0] for pt in allPoints]
        yS = [pt[1] for pt in allPoints]
        shape = (int(max(yS) + 100), int(max(xS) + 100))
        zeros = np.zeros(shape, np.uint8)
        changeContour = np.array([self.changePoints], dtype=np.int32)
        otherContour = np.array([change.changePoints], dtype=np.int32)
        changeIm = cv2.drawContours(zeros.copy(), changeContour, -1, 255, -1)
        otherIm = cv2.drawContours(zeros.copy(), otherContour, -1, 255, -1)
        return cv2.bitwise_and(changeIm, otherIm).any()
    
    def __eq__(self, other):
        if self.changeType in [None, "delete"]:
            return other is None or other.changeType in [None, "delete"]
        else:
            return \
            (other is not None and other.changeType is not None) and \
            (other.center[0] - 20 < self.center[0] < other.center[0] + 20) and \
            (other.center[1] - 20 < self.center[1] < other.center[1] + 20) and \
            (other.width - 20 < self.width < other.width + 20) and \
            (other.height - 20 < self.height < other.height + 20) and self.changeOverlap(other)
    
    def __repr__(self):
        return f"({self.changeType}) {self.center} by {self.width},{self.height}"
    
    def beforeAfter(self):
        return hStackImages([self.before, self.after])

In [5]:
@dataclass
class Camera:
    camNum: int
    activeZone: list
    
    IMAGE_BUFFER_DEPTH = 7
    CAPTURE_FRAMES = 3
    MILLIMETERS_PER_PIXEL = 2
    MI = 2
    xmax = 2560
    ymax = 1920

    def __post_init__(self):
        self.imageBuffer = [None for i in range(self.IMAGE_BUFFER_DEPTH)]
        self.referenceFrame = None
        self.baseFrame = None
        self.changes = []
        self.setActiveZone(self.activeZone)
        self.setReferenceFrame()
        self.setBaseFrame()
    
    @property
    def mostRecentFrame(self):
        return self.imageBuffer[0]
    
    def setBaseFrame(self):
        self.baseFrame = self.mostRecentFrame
    
    def setReferenceFrame(self):
        self.referenceFrame = self.imageBuffer[1] if self.imageBuffer[1] is not None else self.imageBuffer[0]
    
    def setActiveZone(self, newAZ):
        self.activeZone = np.float32(newAZ)
    
    def pointInActiveZone(self, p):
        return cv2.pointPolygonTest(self.activeZone, p, False) >= 0
    
    def collectImage(self) -> np.ndarray:
        cap = cv2.VideoCapture(self.camNum)
        sleep(delay)
        image = None
        try:
            for frame in range(self.CAPTURE_FRAMES):
                ret, cv2_im = cap.read()
                sleep(delay)
            image = sr.upsample(cv2_im)
            self.imageBuffer.insert(0, image)
            self.imageBuffer.pop()
        except Exception as e:
            print(f"Failed to capture Camera: {e}")
        finally:
            cap.release()
        return image
    
    @staticmethod
    def contoursBetween(im0, im1):
        if im0 is None or im1 is None:
            return []
        img_height = im0.shape[0]
        diff = cv2.absdiff(cv2.cvtColor(im0, cv2.COLOR_BGR2GRAY),
                           cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY))
        thresh = cv2.threshold(diff,82,255,cv2.THRESH_BINARY)[1]
        kernel = np.ones((3, 3), np.uint8) 
        dilate = cv2.dilate(thresh, kernel, iterations=2)
        return cv2.findContours(dilate.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    
    def changeBetween(self, changeFrame, referenceFrame):
        maskedRefFrame = self.maskFrameToActiveZone(referenceFrame)
        maskedChangeFrame = self.maskFrameToActiveZone(changeFrame)
        contours = self.contoursBetween(maskedRefFrame, maskedChangeFrame)
        
        newIm = changeFrame.copy()
        oldIm = referenceFrame.copy()
        boxes = []
        largeContours = []
        newClips = []
        oldClips = []
        
        filteredContours = []
        for contour in contours:
            bRect = cv2.boundingRect(contour)
            x, y, w, h = bRect
            area = w * h
            if area > 1000:
                filteredContours.append(contour)
        if len(filteredContours) > 0:
            allPoints = np.array([pt for c in filteredContours for d in c for pt in d], dtype="float32") 
            return CameraChange(self.camNum, allPoints, oldIm, newIm, changeType="unclassified")
        else:
            return CameraChange(self.camNum, None, None, None, changeType=None)
    
    def referenceFrameDelta(self):
        return self.changeBetween(self.mostRecentFrame, self.referenceFrame)
    
    @staticmethod
    def swapBox(srcIm, dstIm, box):
        swapped = dstIm.copy()
        x, y, w, h = box
        orig = srcIm[y:y+h, x:x+w]
        swapped[y:y+h, x:x+w] = orig
        return swapped
    
    def changePatchDelta(self, change: CameraChange):
        patched = self.swapBox(self.baseFrame, self.referenceFrame, change.clipBox)
        return self.changeBetween(self.mostRecentFrame, patched)
    
    def changeOverlaps(self, change):
        overlaps = []
        for eC in self.changes:
            if eC.changeOverlap(change) and eC not in overlaps:
                overlaps.append(eC)
        return overlaps

    def classifyChange(self, change: CameraChange):
        overlaps = self.changeOverlaps(change)
        if len(overlaps) == 0:  # Addition
            change.classify("add", CameraChange(None, None, None, None, None))
            return change
        elif len(overlaps) == 1:  # Move or Deletion
            cPD = self.changePatchDelta(overlaps[0])
            if cPD.changeType is not None:  # Move
                patchedOverlaps = [o for o in self.changeOverlaps(cPD) if o != overlaps[0]]
                if len(patchedOverlaps) != 0:
                    raise Exception(f"Unable to classify change: {change} interacting with {overlaps[0]} (patch has overlaps: {patchedOverlaps}")
                cPD.classify("move", overlaps[0])
                return cPD
            else:  # Deletion
                change.classify("delete", overlaps[0])
                return change
        else:
            raise Exception(f"Unable to classify change: {change}")
    
    def commitChange(self, classifiedChange):
        overlaps = self.changeOverlaps(classifiedChange)
        assert classifiedChange.changeType != "unclassified", "Unable to commit unclassified changes"
        if classifiedChange.changeType == "add":
            print(f"Adding {classifiedChange}")
            self.changes.insert(0, classifiedChange)
        elif classifiedChange.changeType == "move":
            print(f"Moving {classifiedChange.lastChange} to {classifiedChange}")
            self.changes.remove(classifiedChange.lastChange)
            self.changes.insert(0, classifiedChange)
        elif classifiedChange.changeType == "delete":
            print(f"Deleting {classifiedChange.lastChange} with {classifiedChange}")
            self.changes.remove(classifiedChange.lastChange)
        else:
            raise Exception(f"Unrecognzed changeType: {classifiedChange}")
    
    def capture(self):
        return self.collectImage()
    
    def cropToActiveZone(self, image):
        pts = np.int32(self.activeZone)
        mask = np.zeros(image.shape[:2], np.uint8)
        cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)
        dst = cv2.bitwise_and(image, image, mask=mask)
        return dst
        
    def drawActiveZone(self, image):
        pts = np.int32(self.activeZone)
        azOverlaidImage = image.copy()
        return cv2.polylines(azOverlaidImage, [pts], isClosed=True, color=(0,255,0), thickness=5)

    def maskFrameToActiveZone(self, frame=None):
        frame = self.mostRecentFrame if frame is None else frame
        mask = np.zeros((frame.shape[:2]), dtype="uint8")
        masked = cv2.fillPoly(mask, [np.array(self.activeZone, np.int32)], 255)
        return cv2.bitwise_and(frame, frame, mask=masked)
    
    def maskFrameToNonActiveZone(self, frame=None):
        frame = self.mostRecentFrame if frame is None else frame
        zeroes = np.ones((frame.shape[:2]), dtype="uint8")
        masked = cv2.fillPoly(zeroes, [np.array(self.activeZone, np.int32)], 0)
        return cv2.bitwise_and(frame, frame, mask=masked)

    @classmethod
    def drawBoxesOnImage(cls, image, boxes, color=(0,0,255)):
        imageWithBoxes = image.copy()
        for x, y, w, h in boxes:
            cv2.rectangle(imageWithBoxes, (x, y), (x+w, y+h), color, 2)
            presumedBasePoint = [x + int(w/2), y + int(h/2)]
            cv2.circle(imageWithBoxes, presumedBasePoint, radius=4, thickness=4, color=(0,255,255))
            cv2.putText(imageWithBoxes, f'{w*h}p-[{x}-{x+w}, {y}-{y+w}]', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (255, 0, 0), 2, cv2.LINE_AA)
        return imageWithBoxes
    
    @staticmethod
    def getAngle(pt0, pt1, pt2):
        u = [pt1[0] - pt0[0], pt1[1] - pt0[1]]
        v = [pt2[0] - pt0[0], pt2[1] - pt0[1]]

        duv = u[0] * v[0] + u[1] * v[1]
        mu = (u[0] ** 2 + u[1] ** 2) ** 0.5
        mv = (v[0] ** 2 + v[1] ** 2) ** 0.5
        return degrees(acos( duv / (mu * mv) ))

    @classmethod
    def calibrationTriangleToRealCoordinates(cls, triPts):
        angles = {}
        for idx, pt in enumerate(triPts):
            vectors = []
            otherPts = [op for op in triPts if not (pt == op).all()]
            angles[idx] = cls.getAngle(pt, *otherPts)

        ninetyPt = triPts[sorted([(key, abs(90 - angle)) for key, angle in angles.items()], key=lambda x: x[1])[0][0]]
        sixtyPt = triPts[sorted([(key, abs(60 - angle)) for key, angle in angles.items()], key=lambda x: x[1])[0][0]]
        thirtyPt = triPts[sorted([(key, abs(30 - angle)) for key, angle in angles.items()], key=lambda x: x[1])[0][0]]

        ninetyThirtyDiff = [d1 - d0 for d1, d0 in zip(thirtyPt, ninetyPt)]
        missingPt = np.float32([pt + d for pt, d in zip(sixtyPt, ninetyThirtyDiff)])

        cameraPts = np.float32([ninetyPt, sixtyPt, missingPt, thirtyPt])
        # These points are in a MM / 2 scale
        realPts = np.float32([[500, 500], [538.5, 500], [538.5, 525], [500, 525]])

        return cameraPts, realPts

    def tuneToCalibrationBox(self, observedCorners, unwarpedCorners):
        self.M = cv2.getPerspectiveTransform(observedCorners, unwarpedCorners)

    def calibrate(self, trianglePts):
        cameraPts, realPts = self.calibrationTriangleToRealCoordinates(trianglePts)
        self.tuneToCalibrationBox(cameraPts, realPts)

    def convertCameraToRealSpace(self, p):
        assert not (self.M is None), "Must calibrate camera before converting coordinates"
        M = self.M
        px = (M[0][0]*p[0] + M[0][1]*p[1] + M[0][2]) / ((M[2][0]*p[0] + M[2][1]*p[1] + M[2][2]))
        py = (M[1][0]*p[0] + M[1][1]*p[1] + M[1][2]) / ((M[2][0]*p[0] + M[2][1]*p[1] + M[2][2]))
        return (px, py)
    
    def cameraPointsToRealDistance(self, pt0, pt1):
        """ Calculates distance between cam's RealSpace coordinates to distance in mm """
        ptr0 = cam0.convertCameraToRealSpace(pt0)
        ptr1 = cam0.convertCameraToRealSpace(pt1)
        return distanceFormula(ptr0, ptr1) * self.MILLIMETERS_PER_PIXEL
        
    def showUnwarpedImage(self):
        warp = cv2.warpPerspective(self.cropToActiveZone(self.mostRecentFrame), self.M, (1000, 1000))
        f, ax = plt.subplots(1, 1, figsize=(10, 10))
        ax.imshow(warp)
        plt.show()
        return warp

In [6]:
@dataclass
class RemoteCamera(Camera):
    address: str
    rotate: bool = False

    def collectImage(self) -> np.ndarray:
        image = None
        try:
            for i in range(5):
                resp = requests.get(self.address, stream=True).raw
                image = np.asarray(bytearray(resp.read()), dtype="uint8")
                image = cv2.imdecode(image, cv2.IMREAD_COLOR)
                if self.rotate:
                    image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
                if image is not None:
                    break
            assert image is not None, f"Failed to collect image for Camera {self.camNum}"
            
            self.imageBuffer.insert(0, image)
            self.imageBuffer.pop()
        except Exception as e:
            print(f"Failed to capture Camera: {e}")
        return image

In [7]:
@dataclass
class CaptureConfiguration:
    name: str
    
    def __post_init__(self):
        self.lastCapture = None
    
    def capture(self):
        self.lastCapture = {cam.camNum: cam.capture() for cam in cameras.values()}

    def setBase(self):
        for c in cameras.values():
            c.setBaseFrame()
    
    def setReference(self):
        for c in cameras.values():
            c.setReferenceFrame()
    
    def scanCamsForCalibrationBoxes(self):
        return {cam.camNum: (qrs := decode(cam.mostRecentFrame), len(qrs)) for cam in cameras.values()}

    def unwarpedOverlaidCameras(self):
        im = None
        for camNum, cam in cameras.items():
            warp = cv2.warpPerspective(cam.cropToActiveZone(cam.mostRecentFrame), cam.M, (1000, 1000))
            im = warp if im is None else cv2.addWeighted(im, 0.6, warp, 0.3, 0)
        return im
    
    def saveConfiguration(self):
        state = {
            camNum: {
                "addr": cam.address,
                "rot": cam.rotate,
                "az": json.dumps(cam.activeZone.tolist()),
                "mw": cam.minimumWidth,
                "m": json.dumps(cam.M.tolist()) if cam.M is not None else None}
            for camNum, cam in cameras.items()}
        state["pov"] = {
            "addr": pov.address,
            "rot": pov.rotate
        }
        with open("observerConfiguration.json", "w") as f:
            f.write(json.dumps(state, indent=2))
            
    def recoverConfiguration(self):
        with open("observerConfiguration.json", "r") as f:
            state = json.loads(f.read())
        for camNum, camDef in state.items():
            if str(camNum) == "pov":
                global pov
                addr = camDef['addr']
                rot = camDef['rot']
                pov = RemoteCamera(address=addr, rotate=rot, activeZone=[], camNum=-99)
            else:
                addr = camDef['addr']
                az = np.float32(json.loads(camDef['az']))
                cameras[int(camNum)] = RemoteCamera(address=addr, activeZone=az, camNum=int(camNum))
                cameras[int(camNum)].M = None if camDef['m'] is None else np.float32(json.loads(camDef['m']))

In [8]:
@dataclass
class Capture:
    cid: str
    changeSet: dict[int, CameraChange]
    
    def __eq__(self, other):
        if len(self.changeSet) != len(other.changeSet):
            return False

        for camNum, change in self.changeSet.items():
            if camNum not in other.changeSet or change != other.changeSet[camNum]:
                return False
        return True
    
    def changeOverlap(self, camNum: int, change: CameraChange):
        if camNum not in self.changeSet:
            return False
        
        cam = cameras[camNum]
        camChange = self.changeSet[camNum]
        changeContour = np.array([camChange.changePoints], dtype=np.int32)
        otherContour = np.array([change.changePoints], dtype=np.int32)
        zeros = np.zeros(cam.mostRecentFrame.shape[:2], np.uint8)
        changeIm = cv2.drawContours(zeros.copy(), changeContour, -1, 255, -1)
        otherIm = cv2.drawContours(zeros.copy(), otherContour, -1, 255, -1)
        return cv2.bitwise_and(changeIm, otherIm).any()
    
    def visual(self):
        return hStackImages([cs.after for cs in self.changeSet.values()])
    
    def realCoordinates(self):
        return [cameras[camNum].convertCameraToRealSpace(change.center)
                for camNum, change in self.changeSet.items()]
    
    @property
    def realCenter(self):
        centers = self.realCoordinates()
        midIndex = int(len(centers) / 2)
        return [sorted(coord)[midIndex] for coord in list(zip(*centers))]


class CaptureMemory:
    def __init__(self, captures=None):
        self.captures = captures if captures is not None else {}
        
    def buildMiniMap(self):
        objs = list(self.captures.values())
        image = np.zeros([1000, 1000], dtype="uint8")
        if not objs:
            return image

        xS = [o.realCenter[0] for o in objs]
        yS = [o.realCenter[1] for o in objs]
        minX = int(max(min(xS) - 10, 0))
        maxX = int(min(max(xS) + 10, 1000))
        minY = int(max(min(yS) - 10, 0))
        maxY = int(min(max(yS) + 10, 1000))
        dim = [maxX - minX, maxY - minY]

        for obj in objs:
            cv2.circle(image, np.array(obj.realCenter, dtype="uint32"), radius=1, color=255, thickness=-1)
        return image[minY:maxY, minX:maxX]
        
    def memorize(self, changeSet):
        captureInteractions = []
        for cam, changes in changeSet.items():
            for change in changes:
                for capture in self.captures.values():
                    if capture not in captureInteractions and capture.changeOverlap(cam, change):
                        captureInteractions.append(capture)
                        break
        maxChangesPerCam = max([len(changes) for changes in changeSet.values()])
        # If changeSet has 1 change or less per camera and no interactions -- new Capture
        if len(captureInteractions) == 0 and maxChangesPerCam == 1:
            captureSet = {camNum: change
                          for camNum, changes in changeSet.items()
                          for change in changes}
            objectName = f"obj{len(self.captures)}"
            self.captures[objectName] = Capture(objectName, captureSet)
            print(f"Added Object {objectName}")
        # If changeSet has 1 change or less per camera and 1 interactions -- Capture Alter
        elif len(captureInteractions) == 1 and maxChangesPerCam == 1:
            origCapture = captureInteractions[0]
            captureSet = {camNum: change
                          for camNum, changes in changeSet.items()
                          for change in changes}
            origCapture.changeSet = captureSet
            print(f"Altered Object {origCapture.cid} -- {cam} -- {change}")
        # If a changeSet has 2 changes or less per camera and interacts with 1 object leaving 1 shadow -- moved Capture
        elif len(captureInteractions) == 1:
            origCapture = captureInteractions[0]
            origCaptures = origCapture.changeSet
            startPositions = origCapture.realCoordinates()
            startCenter = origCapture.realCenter
            leftOver = {camNum: change
                        for camNum, changes in changeSet.items()
                        for change in changes if camNum not in origCaptures or origCaptures[camNum] != change}
            origCapture.changeSet = leftOver
            endPositions = origCapture.realCoordinates()
            endCenter = origCapture.realCenter
            distances = [distanceFormula(sp, ep) * Camera.MILLIMETERS_PER_PIXEL 
                         for sp in startPositions
                         for ep in endPositions]
            distance = sorted(distances)[int(len(distances) / 2)]
            print(f"Moved {origCapture.cid} from {startCenter} to {endCenter} ({distance} mm)")
        else:
            raise Exception(f"Unable to parse cI: {len(captureInteractions)} || mC: {maxChangesPerCam} || changeSet: {changeSet}\n{format_exc()}")

In [9]:
@dataclass
class ChangeSet:
    changeSet: dict
    
    @property
    def empty(self):
        return sum([change.changeType is None for change in self.changeSet.values()]) == len(self.changeSet)

    def __eq__(self, other):
        return \
            sum([self.changeSet[camNum] == (oCS if (oCS := other.changeSet[camNum]) is not None else None) for camNum in self.changeSet.keys()]) == len(self.changeSet)
    
@dataclass
class TrackedObject(ChangeSet):
    def __repr__(self):
        changeSet = {camNum: cS for camNum, cS in self.changeSet.items() if cS is not None and cS.changeType not in [None, 'delete']}
        return f"TrackedObject({changeSet})"
    
    def previousVersion(self):
        return type(self)({camNum: change.lastChange if change is not None else None for camNum, change in self.changeSet.items()})

In [10]:
class CaptureMachine:
    """ Camera Capture State Machine, primary loop:
                                 {on interaction detection} --> (interaction)            
                                 /                                      |
                __init__ ---> (idle)                                    V
                                  ^------- (observation) <--------- (debounce)
            
    """
    states = ["idle", "interaction", "debounce", "observation", "error"]
    mode = ["terrain", "add_unit", "move_unit"]
    observationThreshold = 3
    def __init__(self, captureConfiguration: CaptureConfiguration):
        self.cc = captureConfiguration
        self.mode = "unit"
        self.cc.capture()
        self.cc.setBase()
        self.cc.setReference()
        self.state = "idle"
        self.interactionDetected = False
        self.debounceCounter = 0
        self.cycleCounter = 0
        self.lastChanges = None
        self.lastClassification = None
        self.memory = []
        
    def __repr__(self):
        return f"CapMac -- {self.state}\nMode: {self.mode}\nID: {self.interactionDetected} | DC: {self.debounceCounter}"
    
    def interactionDetection(self):
        detections = {cam.camNum: cam.interactionDetection(cam.mostRecentFrame) for cam in cameras.values()}
        return sum(detections.values()) > 0
    
    def updateReference(self):
        self.cc.setReference()
        
    def referenceFrameDeltas(self):
        return ChangeSet({camNum: cam.referenceFrameDelta() for camNum, cam in cameras.items()})
    
    def commitChanges(self, objDef):
        try:
            existingIndex = self.memory.index(objDef.previousVersion())
            print(f"Updating Memory {existingIndex}")
            self.memory[existingIndex] = objDef
        except ValueError:
            print(f"New Memory")
            self.memory.append(objDef)
        
        for camNum, change in objDef.changeSet.items():
            cameras[camNum].commitChange(change)
        
        self.lastClassification = None
        self.lastChange = None
        self.cc.setReference()
        
    def cycle(self):
        print(f"Starting Cycle {self.cycleCounter} -- S:{self.state}")
        self.cycleCounter += 1
        self.cc.capture()
        changes = self.referenceFrameDeltas()
        if not changes.empty:
            if self.lastChanges is not None and changes == self.lastChanges:
                print(f"Stable changes. Classifying {changes}!")
                try:
                    objDef = {}
                    for camNum, change in changes.changeSet.items():
                        if change is not None:
                            objDef[camNum] = cameras[camNum].classifyChange(change)
                        else:
                            objDef[camNum] = None
                    objDef = TrackedObject(objDef)
                    print(f"Created classifed ChangeSet: {objDef}")
                    if self.lastClassification == objDef:
                        print(f"Commiting classified ChangeSet: {objDef}")
                        self.commitChanges(objDef)
                    else:
                        self.lastClassification = objDef
                except:
                    raise
            else:
                print("Unstable changes")
                self.lastChanges = changes
        else:
            print("No changes")

In [11]:
cameras = defaultdict(lambda x: None)
pov = None
cc = CaptureConfiguration(__name__)
cc.recoverConfiguration()
cm = CaptureMachine(cc)

In [12]:
if __name__ == "__main__":
    from time import sleep
    while True:
        cm.cycle()
        sleep(1)

Starting Cycle 0 -- S:idle
No changes
Starting Cycle 1 -- S:idle
No changes
Starting Cycle 2 -- S:idle
No changes
Starting Cycle 3 -- S:idle
No changes
Starting Cycle 4 -- S:idle
No changes
Starting Cycle 5 -- S:idle
Unstable changes
Starting Cycle 6 -- S:idle
Stable changes. Classifying ChangeSet(changeSet={0: (unclassified) [982.0, 1033.0] by 66.0,73.0, 2: (unclassified) [426.0, 767.0] by 68.0,61.0, 4: (unclassified) [780.0, 1266.0] by 79.0,68.0})!
Created classifed ChangeSet: TrackedObject({0: (add) [982.0, 1033.0] by 66.0,73.0, 2: (add) [426.0, 767.0] by 68.0,61.0, 4: (add) [780.0, 1266.0] by 79.0,68.0})
Starting Cycle 7 -- S:idle
Unstable changes
Starting Cycle 8 -- S:idle
Stable changes. Classifying ChangeSet(changeSet={0: (unclassified) [985.0, 1034.0] by 68.0,79.0, 2: (unclassified) [421.0, 765.0] by 64.0,48.0, 4: (unclassified) [782.0, 1266.0] by 78.0,72.0})!
Created classifed ChangeSet: TrackedObject({0: (add) [985.0, 1034.0] by 68.0,79.0, 2: (add) [421.0, 765.0] by 64.0,48.0

KeyboardInterrupt: 

In [13]:
cm.memory

[TrackedObject({0: (move) [897.0, 756.0] by 79.0,94.0, 2: (move) [309.0, 1064.0] by 78.0,78.0}),
 TrackedObject({0: (move) [978.0, 1010.0] by 35.0,85.0, 2: (move) [405.0, 803.0] by 67.0,57.0, 4: (add) [802.0, 1236.0] by 82.0,73.0})]

In [17]:
cm.memory[1].previousVersion() == cm.memory[0]

False

In [18]:
cm.memory[0]

TrackedObject({0: (move) [897.0, 756.0] by 79.0,94.0, 2: (move) [309.0, 1064.0] by 78.0,78.0})

In [20]:
cm.memory[1].previousVersion().changeSet[4]

(None) None by None,None